In [2]:
# IMPORTS
from preprocessing_utils import *
from global_vars import *
from random_forest_model import *
from GAN_network_vanila import *
import pandas as pd
from typing import Tuple
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


In [3]:
# get number of unique values in each column
d_unique_vals = {}
d_unique_vals_cnt = {}
d_data_df, d_df_attributes = read_arff_as_dataframe(DIABETES_PATH)
for col in d_data_df.columns:
    d_unique_vals[col] = d_data_df[col].unique()
    d_unique_vals_cnt[col] = len(d_unique_vals[col])

g_unique_vals = {}
g_unique_vals_cnt = {}
g_data_df, g_df_attributes = read_arff_as_dataframe(G_CREDIT_PATH)
for col in g_data_df.columns:
    g_unique_vals[col] = g_data_df[col].unique()
    g_unique_vals_cnt[col] = len(g_unique_vals[col])

In [4]:
# are there any nan values?
print(f"number of nans in diabetes df {d_data_df.isna().values.sum()}")
print(f"number of nans in german df {g_data_df.isna().values.sum()}")

number of nans in diabetes df 0
number of nans in german df 0


In [5]:
# get number of each class
print(f"for diabetes: \n{d_data_df['class'].value_counts()}")
print(f"for german: \n{g_data_df['21'].value_counts()}")

for diabetes: 
b'tested_negative'    500
b'tested_positive'    268
Name: class, dtype: int64
for german: 
b'1'    700
b'2'    300
Name: 21, dtype: int64


In [8]:
print(f"categorical columns: \n{d_data_df.select_dtypes(include=['object']).copy().columns}")
print(f"categorical columns: \n{g_data_df.select_dtypes(include=['object']).copy().columns}")
print(f"numerical columns: \n{d_data_df.select_dtypes(include=['float', 'int']).copy().columns}")
print(f"numerical columns: \n{g_data_df.select_dtypes(include=['float', 'int']).copy().columns}")

categorical columns: 
Index(['class'], dtype='object')
categorical columns: 
Index(['1', '3', '4', '6', '7', '9', '10', '12', '14', '15', '17', '19', '20',
       '21'],
      dtype='object')
numerical columns: 
Index(['preg', 'plas', 'pres', 'skin', 'insu', 'mass', 'pedi', 'age'], dtype='object')
numerical columns: 
Index(['2', '5', '8', '11', '13', '16', '18'], dtype='object')


In [33]:
# to look at the scaled dataframes
from collections import defaultdict
df = d_data_df.copy() # or g_data_df.copy()
d = defaultdict(LabelEncoder)
s = defaultdict(MinMaxScaler)
cols_to_encode = df.select_dtypes(include=['object']).copy().columns
fit = df.apply(lambda x: d[x.name].fit_transform(x) if x.name in cols_to_encode else x)
scaled = fit.apply(lambda x: (x-x.values.min())/(x.values.max()-x.values.min()))

In [45]:
from sklearn.mixture import GaussianMixture

df = pd.DataFrame({'a': [1, 2, 3], 'b': [252.2, 405, 1002], 'c': ['jhon','margorie','clint']})
org_df = df.copy()
cols_to_cluster = df.select_dtypes(include=['float', 'int', 'int64', 'float64']).copy().columns
gmm_per_col = {}
mean_per_col, std_per_col = {}, {}
for col_key in cols_to_cluster:
    gmm_per_col[col_key] = GaussianMixture().fit(df[col_key].values.reshape(-1, 1))
    gmm = gmm_per_col[col_key]
    mean_per_col[col_key], std_per_col[col_key] = gmm.means_, np.sqrt(gmm.covariances_)
gmm_transfomed_df = df.apply(lambda x: (x.values - mean_per_col[x.name].flatten())/(4* std_per_col[x.name].flatten()) if x.name in cols_to_cluster else x)

In [18]:
# for col_key, gmm in gmm_per_col.items():
#     mean_per_col[col_key], std_per_col[col_key] = gmm.means_, np.sqrt(gmm.covariances_)
    # print(f'col:{col_key}, gmm_mean:{gmm.means_}, gmm_std:{np.sqrt(gmm.covariances_)}')

In [28]:
inverse_transformed_df = gmm_transfomed_df.copy()
inverse_transformed_df = inverse_transformed_df.apply(
    lambda x: x.values*4* std_per_col[x.name].flatten() + mean_per_col[x.name].flatten()
    if x.name in gmm_per_col.keys() else x)

In [46]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoders = defaultdict(OneHotEncoder)
categorical_cols = df.select_dtypes(include=['object']).copy().columns
for col_key in categorical_cols:
    one_hot_encoders[col_key].fit(df[col_key].values.reshape(-1, 1))

In [50]:
cols_to_encode = df.select_dtypes(include=['object']).copy().columns

ohe = OneHotEncoder(categories='auto')
feature_arr = ohe.fit_transform(df[cols_to_encode]).toarray()
features_labels = ohe.categories_
dropped_df = df.drop(columns=cols_to_encode, inplace=False)
fitted_df = pd.DataFrame(feature_arr, columns=features_labels)
fitted_df = pd.concat([fitted_df, dropped_df], axis=1)

In [1]:
from preprocessing_utils import *
path_to_arff='Data/diabetes.arff'
data_df, data_attributes = read_arff_as_dataframe(path_to_arff=path_to_arff)
de = DataEncoder()
org_df, data_df = de.prepare_dataframe(data_df)

     preg   plas  pres  skin   insu  mass   pedi   age               class
0     6.0  148.0  72.0  35.0    0.0  33.6  0.627  50.0  b'tested_positive'
1     1.0   85.0  66.0  29.0    0.0  26.6  0.351  31.0  b'tested_negative'
2     8.0  183.0  64.0   0.0    0.0  23.3  0.672  32.0  b'tested_positive'
3     1.0   89.0  66.0  23.0   94.0  28.1  0.167  21.0  b'tested_negative'
4     0.0  137.0  40.0  35.0  168.0  43.1  2.288  33.0  b'tested_positive'
..    ...    ...   ...   ...    ...   ...    ...   ...                 ...
763  10.0  101.0  76.0  48.0  180.0  32.9  0.171  63.0  b'tested_negative'
764   2.0  122.0  70.0  27.0    0.0  36.8  0.340  27.0  b'tested_negative'
765   5.0  121.0  72.0  23.0  112.0  26.2  0.245  30.0  b'tested_negative'
766   1.0  126.0  60.0   0.0    0.0  30.1  0.349  47.0  b'tested_positive'
767   1.0   93.0  70.0  31.0    0.0  30.4  0.315  23.0  b'tested_negative'

[768 rows x 9 columns]
     preg   plas  pres  skin   insu  mass   pedi   age               class
0

ValueError: operands could not be broadcast together with shapes (768,) (5,) 

In [13]:
# ALTERING DATA FILES
import pandas as pd
import numpy as np
from scipy.io import arff as arffIO
import arff

In [15]:
data, meta_data = arffIO.loadarff('Data/diabetes.arff')

In [18]:
data = pd.DataFrame(data)

In [19]:
data = data.apply(lambda col: col.str.decode(encoding='UTF-8') if col.name == 'class' else col)

In [20]:
div_altered_data = data.apply(lambda col: col/2 if col.name != 'class' else col)

In [21]:
rand_altered_data = data.apply(lambda col: col*np.random.random(len(col)) if col.name != 'class' else col)

In [25]:
arff.dump('Data/DIABETES_div_altered_data.arff', div_altered_data.values.tolist(), relation=meta_data.name, names=meta_data._attributes.keys())

In [26]:
arff.dump('Data/DIABETES_rand_altered_data.arff', rand_altered_data.values.tolist(), relation=meta_data.name, names=meta_data._attributes.keys())